In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [14]:
url = r'Data\Clean\2019_airbnb_cleaned.csv'
manhattan_url = r'Data\Clean\2019_manhattan_properties_cleaned.csv'
brooklyn_url = r'Data\Clean\2019_brooklyn_properties_cleaned.csv'


In [15]:
df = pd.read_csv(url)
manhattan_df = pd.read_csv(manhattan_url) 
brooklyn_df = pd.read_csv(brooklyn_url)
scaler = StandardScaler()

In [ ]:
zip_neighborhood_dict = {
# Manhattan
"10001": "Chelsea / Hudson Yards",
"10002": "Lower East Side / Chinatown",
"10003": "East Village / Greenwich Village",
"10004": "Financial District / Battery Park",
"10005": "Financial District",
"10006": "Financial District",
"10007": "Tribeca / Civic Center",
"10009": "Alphabet City / East Village",
"10010": "Gramercy / Stuyvesant Town",
"10011": "Chelsea / West Village",
"10012": "SoHo / NoHo",
"10013": "Tribeca / SoHo",
"10014": "West Village",
"10016": "Murray Hill / Kips Bay",
"10017": "Turtle Bay / Midtown East",
"10018": "Garment District / Midtown",
"10019": "Hell's Kitchen / Midtown West",
"10020": "Rockefeller Center",
"10021": "Upper East Side",
"10022": "Midtown East",
"10023": "Upper West Side",
"10024": "Upper West Side",
"10025": "Morningside Heights / Upper West Side",
"10026": "South Harlem",
"10027": "Harlem",
"10028": "Yorkville",
"10029": "East Harlem",
"10030": "Central Harlem",
"10031": "Hamilton Heights",
"10032": "Washington Heights (South)",
"10033": "Washington Heights (North)",
"10034": "Inwood",
"10035": "East Harlem / Harlem River",
"10036": "Times Square / Theater District",
"10037": "Harlem (East)",
"10038": "Seaport / Civic Center",
"10039": "Harlem (North)",
"10040": "Fort George / Washington Heights",
"10128": "Carnegie Hill",
"10280": "Battery Park City",
"10282": "Battery Park City / Tribeca",
# Brooklyn
"11201": "Brooklyn Heights / Downtown",
"11203": "East Flatbush",
"11204": "Bensonhurst",
"11205": "Clinton Hill / Fort Greene",
"11206": "Williamsburg (South)",
"11207": "East New York",
"11208": "Cypress Hills / East New York",
"11209": "Bay Ridge",
"11210": "Flatbush",
"11211": "Williamsburg (North)",
"11212": "Brownsville",
"11213": "Crown Heights (North)",
"11214": "Bath Beach / Bensonhurst",
"11215": "Park Slope",
"11216": "Crown Heights (South)",
"11217": "Boerum Hill / Gowanus",
"11218": "Kensington",
"11219": "Borough Park",
"11220": "Sunset Park",
"11221": "Bushwick (South)",
"11222": "Greenpoint",
"11223": "Gravesend",
"11224": "Coney Island",
"11225": "Prospect Lefferts Gardens",
"11226": "Flatbush / Ditmas Park",
"11228": "Dyker Heights",
"11229": "Sheepshead Bay",
"11230": "Midwood",
"11231": "Carroll Gardens / Red Hook",
"11232": "Sunset Park (West)",
"11233": "Bedford-Stuyvesant (South)",
"11234": "Mill Basin / Marine Park",
"11235": "Brighton Beach / Manhattan Beach",
"11236": "Canarsie",
"11237": "Bushwick (North)",
"11238": "Prospect Heights",
"11239": "Spring Creek / East New York",
"11249": "Williamsburg (Waterfront)"
}

In [4]:
def get_neighbourhood_from_zip_code(zip):
    try:
        zip = int(zip)
        for key, value in zip_neighborhood_dict.items():
            if (zip == int(key)):
                return value
    except ValueError:
        return None

In [5]:
def get_price_interpretation(value, lower_threshold = 1.5, higher_threshold = 2):
    if abs(value) > lower_threshold and abs(value) < higher_threshold:
        return 'The price is not average'
    elif abs(value) > higher_threshold:
        return 'The price is beyond average'
    else:
        if value < 0 and abs(value) < 0.1:
            return 'Average price'
        elif value > 0 and value < 0.1:
            return 'Average price'
        elif value < 0:
            return 'Lower price average'
        elif value > 0:
            return 'Higher price average'
        else:
            return 'NaN'

In [17]:
df.columns

Index(['id', 'name', 'host_id', 'borough', 'neighbourhood', 'latitude',
       'longitude', 'room_type', 'price', 'minimum_nights',
       'number_of_reviews', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365'],
      dtype='object')

In [29]:
def get_z_prices(dataframe):
    analysis_df = dataframe[['id', 'borough','neighbourhood', 'host_id', 'availability_365']].copy()
    analysis_df['minimum_price'] = dataframe['price']*dataframe['minimum_nights']
    analysis_df['z_value_prices'] = scaler.fit_transform(analysis_df[['minimum_price']])
    analysis_df['z_value_prices'] = analysis_df['z_value_prices']
    analysis_df = analysis_df[(analysis_df['z_value_prices'] >= -2) & (analysis_df['z_value_prices'] <= 2)]
    analysis_df = analysis_df.drop(['z_value_prices'], axis = 1)
    return analysis_df

In [31]:
manhattan_df.columns

Index(['Borough', 'Neighborhood', 'Building Class Category',
       'Tax Class As Of Final Roll 18/1', 'Block', 'Lot', 'Ease-Ment',
       'Building Class As Of Final Roll 18/1', 'Address', 'Apartment Number',
       'Zip Code', 'Residential Units', 'Commercial Units', 'Total Units',
       'Land Square Feet', 'Gross Square Feet', 'Year Built',
       'Tax Class At Time Of Sale', 'Building Class At Time Of Sale',
       'Sale Price', 'Sale Date'],
      dtype='object')

In [32]:
brooklyn_df.columns

Index(['Borough', 'Neighborhood', 'Building Class Category',
       'Tax Class At Present', 'Block', 'Lot', 'Ease-Ment',
       'Building Class At Present', 'Address', 'Apartment Number', 'Zip Code',
       'Residential Units', 'Commercial Units', 'Total Units',
       'Land Square Feet', 'Gross Square Feet', 'Year Built',
       'Tax Class At Time Of Sale', 'Building Class At Time Of Sale',
       'Sale Price', 'Sale Date'],
      dtype='object')

In [ ]:
def get_clean_df(dataframe, city):
    dataframe = dataframe[['borough','Zip Code', 'Sale Date', 'Sale Price', '']].copy()
    dataframe['Zip Code'] = dataframe['Zip Code'].astype('Int64')
    dataframe['Neighbourhood'] = dataframe['Zip Code'].apply(get_neighbourhood_from_zip_code)
    dataframe = dataframe.drop(['Zip Code'], axis=1)
    dataframe = dataframe.loc[dataframe['Sale Price'] != 0]
    dataframe['Sale Price'].dropna()
    dataframe['Sale Price'].value_counts()
    return dataframe

In [8]:
def get_sale_price_interpretation(value, lower_threshold = 1.5, higher_threshold = 2):
    if abs(value) > lower_threshold and abs(value) < higher_threshold:
        return 'The price is not average'
    elif abs(value) > higher_threshold:
        return 'The price is beyond average'
    else:
        if value < 0 and abs(value) < 0.3:
            return 'Average price'
        elif value > 0 and value < 0.3:
            return 'Average price'
        elif value < 0:
            return 'Lower price average'
        elif value > 0:
            return 'Higher price average'
        else:
            return 'NaN'

In [9]:
def get_ny_z_prices(dataframe):
    analysis_df = dataframe
    analysis_df['z_value_prices'] = scaler.fit_transform(analysis_df[['Sale Price']])
    analysis_df['z_value_prices'] = analysis_df['z_value_prices']
    analysis_df = analysis_df[(analysis_df['z_value_prices'] >= -2) & (analysis_df['z_value_prices'] <= 2)]
    #analysis_df['z_interpretation'] = analysis_df['z_value_prices'].apply(get_sale_price_interpretation)
    return analysis_df

In [10]:
def get_clean_new_york_df():
    new_york_df = pd.concat([get_clean_df(manhattan_df,'manhattan'),get_clean_df(brooklyn_df,'brooklyn')])
    new_york_df = get_ny_z_prices(new_york_df)
    return new_york_df

In [11]:
def main():
    final_df = get_z_prices(df)
    final_df.to_csv('NYC_Airbnb_2019.csv')
    new_york_2019_df = get_clean_new_york_df()
    new_york_2019_df.to_csv('NYC_Housing_2019.csv')

In [16]:
main()

KeyError: "['neighbourhood_group', 'host_name'] not in index"